In [1]:
# For passwords & database access:
import psycopg2
import psycopg2.extras
import getpass

# For plotting
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import display
import ipywidgets

# For math
import numpy as np
import pandas as pd
from decam_utils import *
from scipy.optimize import curve_fit
from scipy.stats import linregress

import time

In [2]:
# Logging in
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")
db = psycopg2.connect( f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'" )

db.autocommit = True # Makes the computer nicer about SQL syntax errors

cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

DB User:  kennedyt
DB Password:  ············


In [3]:
minnumdet = 5 # Minimum number of detections on the first night of data for the filter to consider a candidate
maxmgerr = 0.03 # Maximum median mag error (In each candidate-night-filter)
numsig = 3 # How many sigma is a 'detection'?
magamp = 0.0 # Minimum magnitude change throughout the candidates first night of detection

In [4]:
# Get a list of every MJD for which we have an image
query = ("SELECT e.mjd FROM exposures e "
         "WHERE (e.proposalid = '2021A-0113' "
         "OR e.proposalid = '2021B-0149') "
         "AND (e.ra < 50 "
         "OR e.dec > 0) "
         "LIMIT 10000000")
cursor.execute( query, )

datesquery = np.array(cursor.fetchall())

In [5]:
# Make a list of every observing night (by rounding to the nearest MJD)
odatesrd = np.unique( np.round( datesquery ) )
datemsks = [np.where(np.abs(datesquery-i) < 0.5) for i in odatesrd]
odates = np.array([np.mean(datesquery[i]) for i in datemsks])

In [6]:
def lin(x,m,b):
    return m*x + b

In [7]:
fr = []
frdates = []
filidx = []
fitpararr = []
amps = []
maxmags = []
medmgerrs = []
slopeerrs = []
numdets = []

t1 = time.time() # should be about 30 seconds

goodfnms = good_fnms(cursor)[0] # Pulls all the exposure IDs of science images 

for field in ["COSMOS", "ELIAS"]:
    for fil in ["g","r","i"]:
        tempfr = [] # List for the names of fast-rising candidates
        tempfrdates = [] # List for the dates on which fast riser behavior was observed
        
        oldcands = np.array( [] ) # used to check if candidates have been detected before

        for i in range(len(odatesrd)):
            # Loop through each observing night
            # Grab all detections within a day of the night in question
            lodate = odatesrd[i] - 0.5
            hidate = odatesrd[i] + 0.5
            if field == "COSMOS":
                query = ("SELECT o.candidate_id, o.id, e.mjd, o.mag, rbs.rb, e.id, o.magerr FROM objects o "
                         "JOIN subtractions s ON s.id=o.subtraction_id "
                         "JOIN exposures e on s.exposure_id = e.id "
                         "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 "
                         "WHERE q3c_radial_query(e.ra,e.dec,150,2.2,3) "
                         "AND rbs.rb > 0.4 "
                         "AND e.mjd > %s "
                         "AND e.mjd < %s "
                         "AND e.filter = %s "
                         "AND (e.proposalid = '2021A-0113' "
                         "OR e.proposalid = '2021B-0149') "
                         "LIMIT 10000000")
            elif field == "ELIAS":
                query = ("SELECT o.candidate_id, o.id, e.mjd, o.mag, rbs.rb, e.id, o.magerr FROM objects o "
                         "JOIN subtractions s ON s.id=o.subtraction_id "
                         "JOIN exposures e on s.exposure_id = e.id "
                         "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 "
                         "WHERE q3c_radial_query(o.ra,o.dec,8.5,-43.5,2) "
                         "AND rbs.rb > 0.4 "
                         "AND e.mjd > %s "
                         "AND e.mjd < %s "
                         "AND e.filter = %s "
                         "AND (e.proposalid = '2021A-0113' "
                         "OR e.proposalid = '2021B-0149') "
                         "LIMIT 10000000")
            cursor.execute( query, ( lodate, hidate, fil, ) )

            # Store that nights detections in an array
            tempres = np.array( cursor.fetchall(), dtype=str ).transpose()

            if len( tempres ) > 0: # Takes care of errors from gaps in the observations by skipping empty nights
                # Cut out duplicate objects
                dupearr = np.array( [tempres[0], tempres[2], tempres[3], tempres[4], tempres[5], tempres[6]] ).transpose()
                dupearr = [ " ".join(j) for j in dupearr ]
                dupearr, ind = np.unique( dupearr, return_index=True )

                # Rejoin the array without the duplicates
                uarr = np.array( [ i.split(" ") for i in dupearr ] ).transpose()
                res = np.array( [uarr[0], tempres[1][ind], uarr[1], uarr[2], uarr[3], uarr[4], uarr[5]] )
                
                # Cut out non-science images
                msk = np.isin(res[5], goodfnms)
                res = np.array([i[msk] for i in res])
                
                # Create an array of each unique candidate detected that night
                ucands, uind, ucounts = np.unique( res[0], return_counts=True, return_index=True )
                
                # Isolate those with at least numdet detections
                goodcands = ucands[ ucounts >= minnumdet ]


                # Make an array of detections for each of those candidates, store all of those arrays in "arr"
                arr = np.ones( len( goodcands ), dtype=object )
                for j in range( len( goodcands ) ):
                    arr[j] = res[:,res[0,:] == goodcands[j]]
                
                # Mask out those that 
                # A) Don't change by at least magamp mags (currently minimum is 0)
                # B) Have a median mag error of > 0.03
                # C) Don't have a rise with at least 3-sigma confidence
                good = np.ones( len( arr ), dtype=bool )
                for j in range( len( arr ) ):
                    # A)
                    amp = np.max( arr[j][3].astype(float) ) - np.min( arr[j][3].astype(float) )
                    if amp < magamp:
                        good[j] = False
                    else:
                        ntmjds  = arr[j][2].astype(float)
                        ntmags  = arr[j][3].astype(float)
                        ntmgerr = arr[j][6].astype(float)
                        msk1 = ~np.isnan(ntmags)
                        msk2 = ~np.isnan(ntmgerr)
                        msk = msk1 & msk2
                        # print(np.unique([msk,msk1]))
                        ntmjds, ntmags, ntmgerr  = ntmjds[msk], ntmags[msk], ntmgerr[msk]
                        
                        maxmag = np.min(ntmags)
                        medmgerr = np.median(ntmgerr)
                        
                        # B
                        if medmgerr > maxmgerr:
                            good[j] = False
                        
                        elif arr[j][0][0] in oldcands:
                            good[j] = False
                        # C
                        else:
                            # Using scipy curve_fit
                            # fitpars = curve_fit(lin, ntmjds, ntmags, sigma=ntmgerr )
                            # slope, intercept, slopeerr = fitpars[0][0], fitpars[0][1], fitpars[1][0][0]
                            # if slope + np.abs(slopeerr) > 0:
                            #     good[j]=False

                            # Using np.polyfit, weights, and error in params:
                            fitpars = np.polyfit(ntmjds, ntmags, 1, w=1/ntmgerr, cov=True)
                            slope, intercept = fitpars[0]
                            slopeerr = fitpars[-1][0][0]
                            numdet = len(ntmjds)
                            if slope + numsig * np.abs(slopeerr) > 0:
                                good[j] = False
                            if good[j] == False:
                                pass

                            # Using linregress and Pearson:
                            # fitpars = linregress(ntmjds, ntmags)
                            # slope, intercept, pearson = fitpars[0], fitpars[1], fitpars[2]
                            # if (pearson > -0.5) | (np.isnan(pearson)):
                            #     good[j] = False

                            else:
                                # Save the parameters of the good ones
                                fitpararr.append([slope,intercept])
                                amps.append(amp)
                                maxmags.append(maxmag)
                                medmgerrs.append(medmgerr)
                                slopeerrs.append(slopeerr)
                                numdets.append(numdet)
                    
                # Apply those cuts and append the candidates that passed to fr and frdates
                for j in range( len( arr[good] ) ):
                    tempfr.append( arr[good][j][0][0] )
                    tempfrdates.append( [lodate, hidate] )
            
            # Make a note of all candidates detected this night (for cutting out things that have been previously detected)
            try:
                oldcands = np.append( oldcands, ucands )
            except NameError:
                pass
            fr.append(tempfr)
            frdates.append(tempfrdates)
        filidx.append([len(j) for j in fr][-1])
        
x = []
y = []
ind = np.unique(fr, return_index=True)[1]
for i in [fr[ind] for ind in sorted(ind)]:
    x.extend(i)
for i in [frdates[ind] for ind in sorted(ind)]:
    y.extend(i)
fr = x
frdates = y

fitpararr = np.array(fitpararr)
amps = np.array(amps)
maxmags = np.array(maxmags)

t2 = time.time()
print(t2-t1)

del x, y, ind

35.56601166725159


/global/common/software/nersc/cori-2022q1/sw/python/3.9-anaconda-2021.11/lib/python3.9/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [8]:
### Checks whether each passed candidate's night of fast rise was on it's first or only night

fn = []
on = []
for i in range(len(fr)):
    query = ("SELECT o.mag, e.mjd FROM objects o "
             "JOIN subtractions s ON s.id=o.subtraction_id "
             "JOIN exposures e on s.exposure_id = e.id "
             "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 "
             "WHERE rbs.rb > 0.4 "
             "AND (e.proposalid = '2021A-0113' "
             "OR e.proposalid = '2021B-0149') "
             "AND o.candidate_id = %s "
             "ORDER BY e.mjd "
             "LIMIT 10000000")
    cursor.execute( query, ( fr[i], ) )
    tempres = np.array( cursor.fetchall(), dtype=float ).transpose()
    if len(tempres) == 0:
        fn.append(False)
        on.append(False)
    else:
        lastnight = float(tempres[1][-1])
        firstnight = float(tempres[1][0])
        if firstnight < frdates[i][0]:
            fn.append(False)
            on.append(False)
        else:
            fn.append(True)
            if lastnight > frdates[i][1]:
                on.append(False)
            else:
                on.append(True)

In [9]:
len(fr), len(fitpararr)

(21, 21)

In [10]:
### Combines all of the information collected above into one big dataframe
### Each candidate gets at least one row (cands that pass in multiple filters get more rows)

df = pd.DataFrame(index = range(len(fr)), columns = ["CandID", "Field", "Filter", "Night", "DelMag", "PeakMag", "FirstNight", "OnlyNight", "Slope", "Intercept", "MedErr"])

for i in range(len(fr)):
    df["CandID"][i] = fr[i]
    if i < sum(filidx[:3]):
        df["Field"][i] = "COSMOS"
    else: 
        df["Field"][i] = "ELAIS"
    df["Night"][i] = frdates[i][0]+0.5
    df["Slope"][i] = fitpararr[i][0]
    df["Intercept"][i] = fitpararr[i][1]
    df["DelMag"][i] = amps[i]
    df["PeakMag"][i] = maxmags[i]
    df["MedErr"][i] = medmgerrs[i]
    df["FirstNight"][i] = fn[i]
    df["OnlyNight"][i] = on[i]
    if i < filidx[0]:
        df["Filter"][i] = "g"
    elif (i >= filidx[0]) & (i < sum(filidx[:2])):
        df["Filter"][i] = "r"
    elif (i >= sum(filidx[:2])) & (i < sum(filidx[:3])):
        df["Filter"][i] = "i"
    elif (i >= sum(filidx[:3])) & (i < sum(filidx[:4])):
        df["Filter"][i] = "g"
    elif (i >= sum(filidx[:4])) & (i < sum(filidx[:5])):
        df["Filter"][i] = "r"
    elif (i >= sum(filidx[:5])): #& (i < sum(filidx[:6])):
        df["Filter"][i] = "i"
### This takes care of duplicates that arise from having observations closer together than usual
df = df.drop_duplicates(subset=["CandID", "Field", "Filter", "DelMag", "PeakMag"])
try:
    df = df.reset_index(drop=True)
except ValueError:
    pass
fr = df["CandID"]
frdates = [[df["Night"][i]-0.5,df["Night"][i]+0.5] for i in range(len(fr))]

In [11]:
### Summary of Results:
print("There were %s total first-night fast risers" % len(df))
print("Spread over %s candidates (repeats mean detection in more than one filter)" % len(np.unique(df["CandID"])))
print("%s in g, %s in r, and %s in i" % (len(df[df["Filter"]=="g"]), len(df[df["Filter"]=="r"]), len(df[df["Filter"]=="i"])))
print("%s in COSMOS, %s in ELAIS" % (len(df[df["Field"]=="COSMOS"]), len(df[df["Field"]=="ELAIS"])))
# print("%s of these rose quickly on their first night" % len(df[df["FirstNight"]==True]))
print("%s of those were only detected on one night" % len(df[df["OnlyNight"]==True]))

There were 21 total first-night fast risers
Spread over 20 candidates (repeats mean detection in more than one filter)
5 in g, 6 in r, and 10 in i
4 in COSMOS, 17 in ELAIS
11 of those were only detected on one night


In [12]:
# clmtmgs, elmtmgs = lmt_mgs(cursor) # limiting magnitudes for plot (don't put too much stock in these)

In [13]:
### Plots of every passing candidate. Probably not necessary, but here if you need it 
### (needs above cell to be run first)

# filts = ["g", "r", "i"]
# all_fshapes = ['o','s','*']
# all_fsizes  = [5, 5, 7]

# for i in df.index:
#     if df["Field"][i] == "COSMOS":
#         all_fcolors = ["darkgreen","red","brown"]
#         lmtmgs = clmtmgs
#     elif df["Field"][i] == "ELAIS": 
#         all_fcolors = ["limegreen","darkorange","peru"]
#         lmtmgs = elmtmgs
#     fig, ax = plt.subplots( 3, 2, figsize=(16,10), sharex='col', sharey=False, gridspec_kw={'width_ratios': [3, 1]} )
#     n=0 # which row of the plot to put labels on
#     for f in range(len(filts)):
#         query = ('SELECT c.id, e.mjd, o.mag, o.magerr, o.ra, o.dec FROM objects o '
#                  'JOIN candidates c ON c.id=o.candidate_id '
#                  'JOIN subtractions s ON s.id=o.subtraction_id '
#                  'JOIN exposures e ON e.id=s.exposure_id '
#                  'JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 '
#                  'WHERE c.id=%s '
#                  'AND e.filter=%s '
#                  'AND rbs.rb > 0.4')
#         cursor.execute( query, ( df["CandID"][i], filts[f], ) )
#         array = np.array( cursor.fetchall() ).transpose()
#         array = rm_dupes(array)
        
#         # Grab data for full lightcurve
#         array2 = plotlc(df["CandID"][i], cursor, show_plot=False)
        
#         ### At left, plot full lightcurve
#         if len(array2) != 0:
#             # msk = np.where((array2[0][f] > df["Night"][i] - 10) & (array2[0][f] < df["Night"][i] + 10))[0]
#             ax[f,0].errorbar( array2[0][f], array2[1][f], yerr=array2[2][f], fmt=all_fshapes[f], ms=all_fsizes[f], alpha=0.8, mew=0, color=all_fcolors[f] )

#             msk = ~np.isin(odatesrd, np.round(array2[0][f]))
#             ylims = ax[f,0].get_ylim()
#             xlims = ax[f,0].get_xlim()
#             ax[f,0].set_xlim(xlims[0],xlims[1])
#             ax[f,0].set_ylim(ylims[0],ylims[1])
#             ax[f,0].plot( odates[msk], lmtmgs[f][msk],'v', ms=5, alpha=0.6, mew=0, color='grey' )
            
            
#         # ### at right, zoom in on one epoch
#         if len(array) > 0:
#             omjds  = array[1].astype(float)
#             omags  = array[2].astype(float)
#             omerrs = array[3].astype(float)
#             ax[f,1].errorbar(omjds, omags, omerrs, fmt=all_fshapes[f], ms=all_fsizes[f]-2, alpha=0.5, mew=0, color=all_fcolors[f])
#             ax[f,0].tick_params(labelbottom=True)
#             ax[f,1].tick_params(labelbottom=True)
            
#             if df["Filter"][i] == filts[f]:
#                 print(array[4][0], array[5][0])
#                 if len(array2[0][f]) != 0:
#                     ax[f,0].set_xlim(array2[0][f][0]-10,array2[0][f][-1]+10)
#                     ax[f,0].set_ylim(np.min(array2[1][f])-0.3,np.max(array2[1][f])+0.3)
#                 # (omjds>df["Night"][i]-0.5) & (omjds<df["Night"][i]+0.5)
#                 mjd = np.median(omjds[(omjds>df["Night"][i]-0.5) & (omjds<df["Night"][i]+0.5)])
#                 mag = np.median(omags[(omjds>df["Night"][i]-0.5) & (omjds<df["Night"][i]+0.5)])
#                 minmag = np.min(omags[(omjds>df["Night"][i]-0.5) & (omjds<df["Night"][i]+0.5)])
#                 maxmag = np.max(omags[(omjds>df["Night"][i]-0.5) & (omjds<df["Night"][i]+0.5)])
                
#                 xarr = np.linspace(mjd-10, mjd+10, 10)
#                 ax[f,1].plot(xarr, lin(xarr, df["Slope"][i], df["Intercept"][i]), alpha=0.3)
#                 ax[f,0].plot(xarr, lin(xarr, df["Slope"][i], df["Intercept"][i]), alpha=0.3)


#                 ax[0,1].set_xlim(mjd-0.3, mjd+0.3)
#                 ax[1,1].set_xlim(mjd-0.3, mjd+0.3)
#                 ax[2,1].set_xlim(mjd-0.3, mjd+0.3)
                
#                 ax[f,1].set_ylim(minmag-0.3, maxmag+0.3)
                
#                 frmsk = np.where((array2[0][f] > mjd-0.5) & (array2[0][f] < mjd+0.5))[0][0]
#                 ax[f,0].errorbar( array2[0][f][frmsk], array2[1][f][frmsk], yerr=array2[2][f][frmsk], fmt=all_fshapes[f], ms=all_fsizes[f], alpha=0.8, mew=0, color="black" )
                
#             ax[f,1].invert_yaxis()
#             ax[f,0].invert_yaxis()
#         else:
#             fig.delaxes(ax[f,0])
#             fig.delaxes(ax[f,1])
#             if f==0:
#                 n=1
            
#     ax[n,0].set_title("Full lightcurve ("+df["CandID"][i]+")")
#     ax[n,1].set_title("Epoch of fast rise")
        
#     plt.show()

#### Overlap with "probably-real" cands
This section checks which of the candidates that passed the first-night fast rise filter were also passed by the "probably-real" cuts described in the paper

In [14]:
bigarr = np.loadtxt('../candidate_nightly_epochs_files/candidate_lightcurves.dat', dtype=str)
prob_real_cands = np.unique(bigarr.transpose()[1]) # list of all "probably-real" candidates
frcands = np.unique(df["CandID"].values) # List of all cands that passed this notebook's filter
# frcands = np.unique(df["CandID"][df["OnlyNight"]==True].values) # List of all single-night cands that passed this notebook's filter
# frcands = np.unique(df["CandID"][df["OnlyNight"]==False].values) # List of all multi-night cands that passed this notebook's filter

In [15]:
prob_not_real_frcands = frcands[~np.isin(frcands,prob_real_cands)]
prob_real_frcands = frcands[np.isin(frcands,prob_real_cands)]

In [16]:
prob_real_frcands

array(['DC21bae', 'DC21efoi', 'DC21fbia', 'DC21kkqh', 'DC21kkte',
       'DC21kldj', 'DC21knzx', 'DC21koer', 'DC21kqjn', 'DC21kvqx',
       'DC21lktc'], dtype=object)

In [17]:
# Why did those that failed fail?
for i in prob_not_real_frcands:
    query = ("SELECT o.candidate_id, o.id, rbs.rb FROM objects o "
             "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 "
             "WHERE o.candidate_id = %s ")
    cursor.execute(query, (i,))
    arr = np.array(cursor.fetchall()).transpose()
    print(i)
    print("Number of detections: {}".format(len(np.unique(arr[2])))) # this *should* take care of dupe objs, as it's very unlikely that two objs would have same rb
    print("Mean RB score: {}".format(np.mean(arr[2].astype(float))))

DC21cxk
Number of detections: 564
Mean RB score: 0.2451875516842105
DC21kkvm
Number of detections: 300
Mean RB score: 0.38097350425324683
DC21kpuo
Number of detections: 11
Mean RB score: 0.6261004636363636
DC21lkjz
Number of detections: 24
Mean RB score: 0.305922232
DC21meps
Number of detections: 10
Mean RB score: 0.65100825
DC21mfjk
Number of detections: 10
Mean RB score: 0.53539458
DC21mfml
Number of detections: 11
Mean RB score: 0.5902085272727273
DC21mfwo
Number of detections: 10
Mean RB score: 0.31848024999999996
DC21mghz
Number of detections: 11
Mean RB score: 0.31518914545454546


#### MP checker
This section generates dates and coords in the formats needed for the MPchecker (https://minorplanetcenter.net/cgi-bin/checkmp.cgi)

In [18]:
from astropy.time import Time
from astropy.coordinates import SkyCoord

In [19]:
for x, i in enumerate(df["CandID"][df["OnlyNight"]==True]):
    query = ("SELECT e.mjd, o.ra, o.dec FROM objects o "
             "JOIN subtractions s ON s.id=o.subtraction_id "
             "JOIN exposures e ON e.id=s.exposure_id "
             "WHERE o.candidate_id = %s "
             "LIMIT 1 ")
    cursor.execute(query,(i,))
    radec = cursor.fetchall()[0]
    datetime = Time(radec[0], format="mjd").iso
    print(i)
    frac = str(round(sum([float(datetime[11:13]), float(datetime[14:16])/60]) / 24, 2))[1:]
    print(datetime[:10]+frac )
    coords = SkyCoord(radec[1], radec[2], unit='deg').to_string('hmsdms')
    print(coords[:2],coords[3:5], coords[6:11])
    print(coords[19:22],coords[23:25], coords[26:31])
    print("")

DC21efoi
2021-04-09.02
09 55 05.66
+02 11 53.05

DC21fbia
2021-04-12.08
10 01 45.17
+02 39 32.59

DC21mfwo
2021-05-27.36
00 29 05.00
-42 13 01.94

DC21mghz
2021-05-27.37
00 33 55.46
-44 29 46.98

DC21kpuo
2021-05-22.36
00 28 50.47
-43 25 36.77

DC21kqjn
2021-05-22.36
00 34 23.79
-43 13 25.25

DC21lkjz
2021-05-22.4
00 33 09.51
-43 44 29.85

DC21lktc
2021-05-22.4
00 31 21.49
-42 54 34.05

DC21meps
2021-05-27.36
00 31 58.60
-43 06 03.33

DC21mfjk
2021-05-27.36
00 31 49.09
-43 08 08.85

DC21mfml
2021-05-27.36
00 30 17.57
-43 19 21.84

